In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings as warn
warn.filterwarnings('ignore')
from IPython.display import Markdown as Md, display as disp
def printmd(string, color='#ce00ce'):
    ccc = "<span style='color:{}'>{}</span>".format(color, string)
    disp(Md(ccc))

In [2]:
from sklearn.tree import DecisionTreeClassifier as DTC, DecisionTreeRegressor as DTR
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
bank = pd.read_csv('bank/bank-full.csv', sep=';')
bank = bank.replace('yes', 1).replace('no', 0)
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0


In [4]:
months = pd.DataFrame({
    'month': bank.month.sort_values().unique(),
    'month_no': [4, 8, 12, 2, 1, 7, 6, 3, 5, 11, 10, 9]
})
bank = bank.merge(months).drop('month', axis=1)
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,duration,campaign,pdays,previous,poutcome,y,month_no
0,58,management,married,tertiary,0,2143,1,0,unknown,5,261,1,-1,0,unknown,0,5
1,44,technician,single,secondary,0,29,1,0,unknown,5,151,1,-1,0,unknown,0,5
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,76,1,-1,0,unknown,0,5
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,92,1,-1,0,unknown,0,5
4,33,unknown,single,unknown,0,1,0,0,unknown,5,198,1,-1,0,unknown,0,5


In [5]:
bank = bank.merge(pd.DataFrame({
    'education': bank.education.unique(),
    'edu': [1, 2, 3, np.nan]
})).drop('education', axis=1)
bank.head()

,age,job,marital,default,balance,housing,loan,contact,day,duration,campaign,pdays,previous,poutcome,y,month_no,edu
0,58,management,married,0,2143,1,0,unknown,5,261,1,-1,0,unknown,0,5,1.0
1,35,management,married,0,231,1,0,unknown,5,139,1,-1,0,unknown,0,5,1.0
2,28,management,single,0,447,1,1,unknown,5,217,1,-1,0,unknown,0,5,1.0
3,42,entrepreneur,divorced,1,2,1,0,unknown,5,380,1,-1,0,unknown,0,5,1.0
4,56,management,married,0,779,1,0,unknown,5,164,1,-1,0,unknown,0,5,1.0


In [6]:
bank = bank.merge(pd.DataFrame({
    'poutcome': bank.poutcome.sort_values().unique(),
    'prev_out': [-1, np.nan, 1, 0]
})).drop('poutcome', axis=1).drop('job', axis=1)
bank.head()

,age,marital,default,balance,housing,loan,contact,day,duration,campaign,pdays,previous,y,month_no,edu,prev_out
0,58,married,0,2143,1,0,unknown,5,261,1,-1,0,0,5,1.0,0.0
1,35,married,0,231,1,0,unknown,5,139,1,-1,0,0,5,1.0,0.0
2,28,single,0,447,1,1,unknown,5,217,1,-1,0,0,5,1.0,0.0
3,42,divorced,1,2,1,0,unknown,5,380,1,-1,0,0,5,1.0,0.0
4,56,married,0,779,1,0,unknown,5,164,1,-1,0,0,5,1.0,0.0


In [7]:
set(bank.contact)

{'cellular', 'telephone', 'unknown'}

In [8]:
bank_dummies = pd.get_dummies(bank)
bank_dummies.sample(10)

#bank_dummies = bank_dummies.dropna()
bank_dummies.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,y,month_no,edu,prev_out,marital_divorced,marital_married,marital_single,contact_cellular,contact_telephone,contact_unknown
0,58,0,2143,1,0,5,261,1,-1,0,0,5,1.0,0.0,0,1,0,0,0,1
1,35,0,231,1,0,5,139,1,-1,0,0,5,1.0,0.0,0,1,0,0,0,1
2,28,0,447,1,1,5,217,1,-1,0,0,5,1.0,0.0,0,0,1,0,0,1
3,42,1,2,1,0,5,380,1,-1,0,0,5,1.0,0.0,1,0,0,0,0,1
4,56,0,779,1,0,5,164,1,-1,0,0,5,1.0,0.0,0,1,0,0,0,1


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    bank_dummies.drop('y', axis=1).drop('prev_out',axis=1).drop('edu',axis=1), # X
    bank_dummies.y,  # y
    test_size=0.2, # porcentaje que será prueba
    random_state=42) # para fijar el aleatorio (reproducibilidad)

In [10]:
grid_parametros = {
    'max_depth':[i for i in range (1,50)],
    'splitter': ['random'],
    'min_samples_split': [15, 20, 30],
}

rf = DTR()

rf_cv = GridSearchCV(
    rf, 
    grid_parametros,
)

In [11]:
rf_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], 'splitter': ['random'], 'min_samples_split': [15, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
rf_cv.best_params_

{'max_depth': 12, 'min_samples_split': 30, 'splitter': 'random'}

In [13]:
rf_cv.best_score_

0.263627514700456

## Observar y Entender

El código que sigue lo utilizamos para saber cómo estaban armandos los valores que ibamos a clasificar y poder entender mejor.

In [14]:
def whatDoIHave(df):
    for i in df.columns:
        printmd('{}:'.format(i))
        printmd("{}".format([i for i in (eval('df.{}.unique()'.format(i)))]), '#ffffff')
        printmd('<br>')

whatDoIHave(bank_dummies)

<span style='color:#ce00ce'>age:</span>

<span style='color:#ffffff'>[58, 35, 28, 42, 56, 39, 49, 51, 29, 48, 32, 38, 60, 59, 46, 36, 27, 47, 45, 33, 52, 57, 40, 54, 31, 55, 50, 43, 41, 34, 30, 37, 44, 53, 23, 25, 24, 26, 20, 22, 62, 74, 61, 21, 78, 66, 71, 80, 77, 64, 67, 81, 63, 73, 72, 65, 70, 89, 76, 68, 75, 69, 82, 83, 19, 79, 18, 95, 86, 87, 90, 94, 84, 92, 85, 88, 93]</span>

<span style='color:#ce00ce'><br></span>

<span style='color:#ce00ce'>default:</span>

<span style='color:#ffffff'>[0, 1]</span>

...

<span style='color:#ffffff'>[0, 1]</span>

<span style='color:#ce00ce'><br></span>

<span style='color:#ce00ce'>contact_unknown:</span>

<span style='color:#ffffff'>[1, 0]</span>

<span style='color:#ce00ce'><br></span>